<a href="https://colab.research.google.com/github/schmuecker/transfer-learning/blob/main/natural_language/text_classification_with_bert/bert_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 5.3 MB/s 
     |████████████████████████████████| 578.0 MB 19 kB/s 
     |████████████████████████████████| 5.9 MB 37.5 MB/s 
     |████████████████████████████████| 438 kB 47.5 MB/s 
     |████████████████████████████████| 1.7 MB 44.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [2]:
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text

# import os
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"


In [3]:
tf.__version__

'2.10.0'

In [4]:
devices = tf.config.experimental.list_physical_devices('GPU')
devices

[]

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(devices[0], True)
    print("Success")
except:
    print("Exception occured")
    pass

Exception occured


**Read more about this dataset here: https://ai.stanford.edu/~amaas/data/sentiment/
As per this article:
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.** 

In [6]:
dataset, info = tfds.load('imdb_reviews', data_dir='./datasets', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling datasets/imdb_reviews/plain_text/1.0.0.incomplete26YSB5/imdb_reviews-train.tfrecord*...:   0%|      …

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling datasets/imdb_reviews/plain_text/1.0.0.incomplete26YSB5/imdb_reviews-test.tfrecord*...:   0%|       …

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling datasets/imdb_reviews/plain_text/1.0.0.incomplete26YSB5/imdb_reviews-unsupervised.tfrecord*...:   0%…

Dataset imdb_reviews downloaded and prepared to ./datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [7]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='./datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        'train': <Sp

In [8]:
dataset

{Split('train'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('test'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('unsupervised'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [9]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [10]:
type(train_dataset)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [11]:
len(train_dataset)

25000

In [12]:
len(test_dataset)

25000

In [13]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [14]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [52]:
X_train = []
y_train = []

for collection in train_dataset:
    input, label = collection
    for sample_input in input:
      X_train.append(sample_input.numpy())
    for sample_label in label:
      y_train.append(sample_label.numpy())

X_train = np.array(X_train)
y_train = np.array(y_train)

X_train.shape, y_train.shape

((25000,), (25000,))

In [16]:
for example, label in train_dataset.take(1):
    print('texts: ', example.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

texts:  [b"A film about wannabee's, never-were's and less-than-heroes making it against all odds. Where have we heard that before. But when the unfortunates are the Shoveller, the Blue Raja and Mr.Furious you know this is not your conventional rags to riches story.<br /><br />A classic performance by Eddie Izzard as Tony P. one of the Disco boys leaders and Geoffrey Rush as Arch Villain shows actual thought went into the casting. <br /><br />Even Greg Kinnear, at first glance an odd choice for the role of Captain Amazing turns out spot on.<br /><br />Watch this film if you're sick of comic-gone-film stereotypes. Why couldn't anger be a super power?"
 b'Helena Bonham Carter is the center of this movie. She plays her role almost immobile in a wheelchair but still brings across her traditional intensity. Kenneth Branagh was tolerable. The movie itself was good not exceptional. If you are a Helena Bonham Carter fan it is worth seeing.'
 b"There's not much to say about this one. Gammera is 

In [17]:
e = tf.keras.layers.experimental.preprocessing.TextVectorization()
e.adapt([
    "I love samosas and jalebi",
    "I love biking and yoga",
    "I love tensorflow"
])

In [18]:
e.get_vocabulary()

['',
 '[UNK]',
 'love',
 'i',
 'and',
 'yoga',
 'tensorflow',
 'samosas',
 'jalebi',
 'biking']

In [19]:
e(["I love pizza"]).numpy()

array([[3, 2, 1]])

In [20]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [21]:
vocab = np.array(encoder.get_vocabulary())
vocab[:25]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but',
       'film', 'on', 'not', 'you', 'are'], dtype='<U14')

In [22]:
example[:2]

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"A film about wannabee's, never-were's and less-than-heroes making it against all odds. Where have we heard that before. But when the unfortunates are the Shoveller, the Blue Raja and Mr.Furious you know this is not your conventional rags to riches story.<br /><br />A classic performance by Eddie Izzard as Tony P. one of the Disco boys leaders and Geoffrey Rush as Arch Villain shows actual thought went into the casting. <br /><br />Even Greg Kinnear, at first glance an odd choice for the role of Captain Amazing turns out spot on.<br /><br />Watch this film if you're sick of comic-gone-film stereotypes. Why couldn't anger be a super power?",
       b'Helena Bonham Carter is the center of this movie. She plays her role almost immobile in a wheelchair but still brings across her traditional intensity. Kenneth Branagh was tolerable. The movie itself was good not exceptional. If you are a Helena Bonham Carter fan it is worth seeing.'],
  

In [23]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  4,  20,  43, ...,   0,   0,   0],
       [  1,   1,   1, ...,   0,   0,   0],
       [213,  22,  73, ...,   0,   0,   0]])

In [24]:
for n in range(3):
    print("Original: ", example[n].numpy())
    print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
    print()

Original:  b"A film about wannabee's, never-were's and less-than-heroes making it against all odds. Where have we heard that before. But when the unfortunates are the Shoveller, the Blue Raja and Mr.Furious you know this is not your conventional rags to riches story.<br /><br />A classic performance by Eddie Izzard as Tony P. one of the Disco boys leaders and Geoffrey Rush as Arch Villain shows actual thought went into the casting. <br /><br />Even Greg Kinnear, at first glance an odd choice for the role of Captain Amazing turns out spot on.<br /><br />Watch this film if you're sick of comic-gone-film stereotypes. Why couldn't anger be a super power?"
Round-trip:  a film about [UNK] [UNK] and [UNK] making it against all [UNK] where have we heard that before but when the [UNK] are the [UNK] the [UNK] [UNK] and [UNK] you know this is not your [UNK] [UNK] to [UNK] [UNK] br a classic performance by [UNK] [UNK] as [UNK] [UNK] one of the [UNK] boys [UNK] and [UNK] [UNK] as [UNK] [UNK] shows 

In [26]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_preprocess/2")
bert_encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_L-6_H-768_A-12/1")

In [27]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.01898347, -0.46152472,  0.3084012 , ...,  0.18858603,
         0.08060566, -0.0900093 ],
       [-0.26919752,  0.09588679, -0.21244422, ...,  0.30378067,
        -0.01326149, -0.08410469]], dtype=float32)>

In [28]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                          
                                                                                                  
 keras_layer_2 (KerasLayer)     {'encoder_outputs':  134734080   ['keras_layer_1[0][0]',      

In [30]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
  3/782 [..............................] - ETA: 1:45:23 - loss: 0.7731 - accuracy: 0.4583

KeyboardInterrupt: ignored

In [ ]:
import sys
print(sys.version)